In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

# Get Device for Training

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cuda device


# Define the Class

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([0], device='cuda:0')


# Model Layers

In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


## nn.Flatten

In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


## nn.Linear

In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


## nn.ReLU

In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1454,  0.4584, -0.4630, -0.1122, -0.2395,  0.4626,  0.0547, -0.0714,
         -0.5643,  0.1935,  0.0365,  0.1475,  0.5033,  0.0562,  0.6769,  0.2395,
          0.0102,  0.2086,  0.1504, -0.0873],
        [ 0.1124,  0.3698, -0.4970,  0.3508,  0.0037,  0.4133,  0.3829, -0.1730,
         -0.2938,  0.0310, -0.7116,  0.3805,  0.4490,  0.1122,  0.4254,  0.4730,
         -0.0056,  0.5414, -0.1288, -0.3266],
        [ 0.1293,  0.4353, -0.6527, -0.0705, -0.1771,  0.1448, -0.1662,  0.0908,
         -0.0400, -0.2007, -0.5470,  0.2546,  0.5988,  0.0985,  0.3172,  0.4339,
          0.0971,  0.4700, -0.2843, -0.2206]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.1454, 0.4584, 0.0000, 0.0000, 0.0000, 0.4626, 0.0547, 0.0000, 0.0000,
         0.1935, 0.0365, 0.1475, 0.5033, 0.0562, 0.6769, 0.2395, 0.0102, 0.2086,
         0.1504, 0.0000],
        [0.1124, 0.3698, 0.0000, 0.3508, 0.0037, 0.4133, 0.3829, 0.0000, 0.0000,
         0.0310, 0.0000, 0.3805, 0.4490, 0.1122, 0.42

## nn.Sequential

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

## nn.Softmax

In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

# Model Parameters

In [12]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0218, -0.0053, -0.0209,  ...,  0.0249, -0.0045,  0.0245],
        [-0.0154, -0.0204, -0.0191,  ...,  0.0196, -0.0357, -0.0017]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0163, -0.0167], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0420, -0.0263, -0.0299,  ...,  0.0393, -0.0417,  0.0236],
        [ 0.0145, -0.0289, -0.0282,  ..., -0.0064, -0.0202,  0.0365]],
       device='cuda:0', grad_fn=<